In [1]:
import pandas as pd

import plotly.offline as py
import plotly.express as px
from plotly.figure_factory import create_table 
import plotly.graph_objects as go

In [2]:
gapminder = px.data.gapminder() # The gapminder dataset

# Create a beautiful table
table = create_table(gapminder.head(10))
py.iplot(table)

In [28]:
child_mortality = pd.read_csv('child_mortality_0_5_year_olds_dying_per_1000_born.csv')

In [27]:
# merge child mortality data with gapminder data
child_mortality_long = child_mortality.melt(id_vars=["country"], var_name="year", value_name="childMortality")
child_mortality_long["year"] = child_mortality_long["year"].astype(int)  # Ensure year is an integer
gapminder_cm = pd.merge(gapminder, child_mortality_long, on=["country", "year"], how="left")

In [29]:
gapminder2007 = gapminder.query("year == 2007")

child_mortality2007 = child_mortality[['country', '2007']]
child_mortality2007 = child_mortality2007.rename(columns={'2007': 'childMortality'})

gapminder2007 = gapminder2007.merge(child_mortality2007, on='country', how='inner')

In [39]:
countries = gapminder['country'].unique()

# Create traces for each country
traces = []
buttons = []

for i, country in enumerate(countries):
    country_df = gapminder[gapminder['country'] == country]
    
    trace = go.Scatter(
        x=country_df['year'],
        y=country_df['lifeExp'],
        mode='lines+markers',
        name=country,
        visible=(country == 'Afghanistan'),  # Only show the first country initially
    )
    traces.append(trace)
    
    # Add button for this country
    button = dict(
        label=country,
        method='update',
        args=[{'visible': [j == i for j in range(len(countries))]},
              {'title': f'Life Expectancy Over Time: {country}'}]
    )
    buttons.append(button)

# Layout with dropdown
layout = go.Layout(
    title='Life Expectancy Over Time: Afghanistan',
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            direction='down',
            pad={"r": 10, "t": 10},
            x=0.0,
            xanchor='left',
            y=1.2,
            yanchor='top',
            showactive=True
        )
    ],
    xaxis=dict(title='Year'),
    yaxis=dict(title='Life Expectancy')
)

fig = go.Figure(data=traces, layout=layout)
fig.show()

In [62]:
# Create a scatter plot (GDP vs Child Mortality)
px.scatter(gapminder_cm,
           x='gdpPercap',
           y='childMortality',
           color='continent',
           size='pop',
           size_max=60,
           hover_name='country',
           animation_frame='year',
           animation_group='country',
           log_x=True,
           range_x=[100, 100000],  # Set x-axis range for better GDP scaling
            range_y=[0, 400],
           labels={'pop': 'Population',
                   'gdpPercap': 'GDP per Capita',
                   'childMortality': 'Child Mortality'}
            )

In [21]:
# Shade the map by Child Mortality using choropleth with projection natural earth
px.choropleth(gapminder_cm,
              locations='iso_alpha',
              color='childMortality', 
              hover_name='country',
              animation_frame='year',
              color_continuous_scale=px.colors.sequential.Plasma,
              projection='natural earth')